In [2]:
"""
Ford Auto ABS HTML Parser + Trigger Metrics Calculator
Extracts monthly performance data and calculates early warning triggers
"""

import re
import json
from pathlib import Path
from bs4 import BeautifulSoup
from typing import Dict, List, Optional
import math

class FordABSParser:
    """Parser for Ford Auto ABS monthly investor reports with trigger analysis"""
    
    def __init__(self, debug=False):
        self.data = []
        self.debug = debug
        
    def parse_html_file(self, filepath: str) -> Optional[Dict]:
        """Parse a single Ford monthly investor report HTML file"""
        
        if self.debug:
            print(f"\n{'='*60}")
            print(f"Parsing: {Path(filepath).name}")
            print('='*60)
        
        try:
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                html_content = f.read()
        except Exception as e:
            print(f"❌ Error reading file: {e}")
            return None
        
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text()
        text = re.sub(r'\s+', ' ', text)
        
        # Extract collection period
        collection_period = self._extract_collection_period(text)
        if not collection_period:
            print("⚠️  Could not extract collection period")
            return None
        
        # Extract all metrics
        metrics = {
            'month': collection_period,
            'filepath': str(filepath),
            'poolBalance': self._extract_pool_balance(text),
            'totalDelinq': self._extract_total_delinquency(text),
            'delinq61plus': self._extract_61plus_delinquency(text),
            'cumLossRatio': self._extract_cumulative_loss_ratio(text),
            'trigger': self._extract_delinquency_trigger(text),
            'numContracts': self._extract_num_contracts(text),
            'transactionMonth': self._extract_transaction_month(text),
        }
        
        if self.debug:
            print(f"\n📊 Extracted Metrics:")
            for key, value in metrics.items():
                if key != 'filepath':
                    status = "✅" if value is not None else "❌"
                    print(f"  {status} {key}: {value}")
        
        # Validate critical fields
        if not all([metrics['poolBalance'], metrics['totalDelinq'], metrics['trigger']]):
            print(f"⚠️  Missing critical data")
            return None
        
        print(f"✅ Successfully parsed!")
        return metrics
    
    def _extract_collection_period(self, text: str) -> Optional[str]:
        """Extract collection period (YYYY-MM format)"""
        # Pattern: "Collection Period November 2025"
        pattern = r'Collection Period\s+(\w+)\s+(\d{4})'
        match = re.search(pattern, text, re.IGNORECASE)
        
        if match:
            month_map = {
                'january': '01', 'february': '02', 'march': '03', 'april': '04',
                'may': '05', 'june': '06', 'july': '07', 'august': '08',
                'september': '09', 'october': '10', 'november': '11', 'december': '12'
            }
            month_name = match.group(1).lower()
            year = match.group(2)
            month_num = month_map.get(month_name)
            
            if month_num:
                return f"{year}-{month_num}"
        
        return None
    
    def _extract_transaction_month(self, text: str) -> Optional[int]:
        """Extract transaction month number"""
        pattern = r'Transaction Month\s+(\d+)'
        match = re.search(pattern, text)
        return int(match.group(1)) if match else None
    
    def _extract_pool_balance(self, text: str) -> Optional[float]:
            """Extract end of period pool balance"""
            # Prefer the pool info table row that includes Adjusted Pool Balance.
            pattern = r'Pool Balance\s*\$\s*([\d,]+\.\d+)\s*\$\s*[\d,]+\.\d+\s*Adjusted Pool Balance'
            match = re.search(pattern, text, re.IGNORECASE)

            if match:
                amount_str = match.group(1).replace(',', '')
                return float(amount_str)

            # Fallback: first Pool Balance dollar amount (less specific).
            fallback = r'Pool Balance\s*\$\s*([\d,]+\.\d+)'
            match = re.search(fallback, text, re.IGNORECASE)

            if match:
                amount_str = match.group(1).replace(',', '')
                return float(amount_str)

            return None
    def _extract_total_delinquency(self, text: str) -> Optional[float]:
        """Extract total delinquency percentage"""
        # Look for "Total Delinquent Receivables" with % of EOP Pool Balance
        pattern = r'Total Delinquent Receivables\s+([\d.]+)\s*%'
        match = re.search(pattern, text)
        
        if match:
            return float(match.group(1))
        
        return None
    
    def _extract_61plus_delinquency(self, text: str) -> Optional[float]:
        """Extract 61+ day delinquency percentage (THE KEY TRIGGER METRIC)"""
        # Ford uses "61+ Delinquent Receivables Balance to EOP Pool Balance"
        pattern = r'61\+ Delinquent Receivables Balance to EOP Pool Balance\s+([\d.]+)\s*%'
        match = re.search(pattern, text)
        
        if match:
            return float(match.group(1))
        
        # Fallback: sum 61-90, 91-120, 120+ buckets
        buckets = [r'61-90 Days Delinquent\s+([\d.]+)\s*%',
                   r'91-120 Days Delinquent\s+([\d.]+)\s*%',
                   r'Over 120 Days Delinquent\s+([\d.]+)\s*%']
        
        total = 0.0
        for bucket_pattern in buckets:
            match = re.search(bucket_pattern, text)
            if match:
                total += float(match.group(1))
        
        return total if total > 0 else None
    
    def _extract_cumulative_loss_ratio(self, text: str) -> Optional[float]:
        """Extract cumulative net loss ratio"""
        pattern = r'Ratio of Cumulative Net Loss.*?Initial Pool Balance\s+([\d.]+)\s*%'
        match = re.search(pattern, text, re.DOTALL)
        
        if match:
            return float(match.group(1))
        
        return None
    
    def _extract_delinquency_trigger(self, text: str) -> Optional[float]:
        """Extract current delinquency trigger percentage based on transaction month"""
        # First get transaction month
        trans_month = self._extract_transaction_month(text)
        if not trans_month:
            return None
        
        # Ford's trigger schedule (from document):
        # 1-12: 0.80%, 13-24: 1.30%, 25-36: 2.40%, 37+: 4.55%
        if trans_month <= 12:
            return 0.80
        elif trans_month <= 24:
            return 1.30
        elif trans_month <= 36:
            return 2.40
        else:
            return 4.55
    
    def _extract_num_contracts(self, text: str) -> Optional[int]:
        """Extract current number of receivables outstanding"""
        pattern = r'Number of Receivables Outstanding.*?End of Period.*?([\d,]+)'
        match = re.search(pattern, text, re.DOTALL)
        
        if match:
            num_str = match.group(1).replace(',', '')
            return int(num_str)
        
        return None
    
    def parse_directory(self, directory: str = "src/components", deal_name: str = "FORD-2025-A") -> List[Dict]:
        """Parse all HTML files in a directory"""
        dir_path = Path(directory)
        html_files = list(dir_path.glob('*.html')) + list(dir_path.glob('*.htm'))
        html_files = sorted(html_files)
        
        if not html_files:
            print(f"❌ No HTML files found in {directory}")
            return []
        
        print(f"Found {len(html_files)} HTML file(s)")
        
        self.data = []
        
        for filepath in html_files:
            metrics = self.parse_html_file(str(filepath))
            
            if metrics:
                metrics['deal'] = deal_name
                self.data.append(metrics)
        
        # Sort by date
        self.data.sort(key=lambda x: x['month'])
        
        print(f"\n✅ Successfully parsed {len(self.data)} file(s)")
        
        return self.data
    
    def calculate_triggers(self) -> List[Dict]:
        """Calculate the 5 trigger metrics for early warning"""
        if len(self.data) < 3:
            print("Need at least 3 months of data for trigger calculations")
            return []
        
        results = []
        
        for i, point in enumerate(self.data):
            R = point['delinq61plus']  # Current 61+ delinquency rate
            Rstar = point['trigger']      # Trigger threshold
            
            # Metric 1: Current Cushion = R*/R - 1
            cushion = (Rstar / R) - 1 if R > 0 else float('inf')
            
            # Metric 2: 3-month slope (linear regression)
            slope = 0.0
            if i >= 2:
                last3 = [(j, (self.data[j]['trigger'] / self.data[j]['delinq61plus']) - 1) 
                         for j in range(i-2, i+1) if self.data[j]['delinq61plus'] > 0]
                
                if len(last3) == 3:
                    n = 3
                    sum_x = sum(p[0] for p in last3)
                    sum_y = sum(p[1] for p in last3)
                    sum_xy = sum(p[0] * p[1] for p in last3)
                    sum_x2 = sum(p[0] * p[0] for p in last3)
                    
                    denominator = (n * sum_x2 - sum_x * sum_x)
                    if denominator != 0:
                        slope = (n * sum_xy - sum_x * sum_y) / denominator
            
            # Metric 3: Volatility of cushion changes
            volatility = 0.0
            if i >= 3:
                cushions = [(self.data[j]['trigger'] / self.data[j]['delinq61plus']) - 1 
                           for j in range(max(0, i-3), i+1) if self.data[j]['delinq61plus'] > 0]
                
                if len(cushions) > 1:
                    deltas = [cushions[j+1] - cushions[j] for j in range(len(cushions)-1)]
                    if deltas:
                        mean = sum(deltas) / len(deltas)
                        variance = sum((d - mean) ** 2 for d in deltas) / len(deltas)
                        volatility = math.sqrt(variance)
            
            # Metric 4: Projected cushions (1, 2, 3 months ahead)
            projected_1m = cushion + 1 * slope
            projected_2m = cushion + 2 * slope
            projected_3m = cushion + 3 * slope
            
            # Metric 5: Breach probability (using normal CDF)
            def norm_cdf(x):
                """Approximation of normal CDF"""
                return 0.5 * (1 + math.erf(x / math.sqrt(2)))
            
            breach_prob_1m = 0.0
            breach_prob_3m = 0.0
            
            if volatility > 0:
                z_1m = (0 - projected_1m) / (volatility * math.sqrt(1))
                z_3m = (0 - projected_3m) / (volatility * math.sqrt(3))
                breach_prob_1m = norm_cdf(z_1m) * 100
                breach_prob_3m = norm_cdf(z_3m) * 100
            
            results.append({
                **point,
                'cushion': cushion,
                'slope': slope,
                'volatility': volatility,
                'projected_1m': projected_1m,
                'projected_2m': projected_2m,
                'projected_3m': projected_3m,
                'breach_prob_1m': breach_prob_1m,
                'breach_prob_3m': breach_prob_3m,
            })
        
        return results
    
    def print_trigger_summary(self, results: List[Dict]):
        """Print trigger analysis summary"""
        if not results:
            return
        
        latest = results[-1]
        
        print("\n" + "="*70)
        print("TRIGGER ANALYSIS - LATEST MONTH")
        print("="*70)
        print(f"Deal: {latest['deal']}")
        print(f"Month: {latest['month']}")
        print(f"Transaction Month: {latest['transactionMonth']}")
        print(f"\n📊 CURRENT STATE:")
        print(f"  61+ Delinquency (R): {latest['delinq61plus']:.4f}%")
        print(f"  Trigger Level (R*): {latest['trigger']:.4f}%")
        print(f"\n🎯 5 KEY METRICS:")
        print(f"  1. Current Cushion: {latest['cushion']:.2f}x")
        print(f"  2. 3-Month Slope: {latest['slope']:.4f} per month")
        print(f"  3. Volatility (σ): {latest['volatility']:.4f}")
        print(f"  4. Projected Cushions:")
        print(f"     - 1 month: {latest['projected_1m']:.2f}x")
        print(f"     - 2 months: {latest['projected_2m']:.2f}x")
        print(f"     - 3 months: {latest['projected_3m']:.2f}x")
        print(f"  5. Breach Probabilities:")
        print(f"     - 1 month: {latest['breach_prob_1m']:.2f}%")
        print(f"     - 3 months: {latest['breach_prob_3m']:.2f}%")
        
        # ALERTS
        print(f"\n🚨 ALERTS:")
        alerts = []
        
        if latest['cushion'] < 1.0:
            alerts.append("🔴 CRITICAL: Cushion below 1.0x")
        elif latest['cushion'] < 2.0:
            alerts.append("🟡 WARNING: Cushion below 2.0x")
        
        if latest['projected_3m'] < 0.5:
            alerts.append("🔴 CRITICAL: 3M projected cushion < 0.5x")
        elif latest['projected_3m'] < 1.0:
            alerts.append("🟡 WARNING: 3M projected cushion < 1.0x")
        
        if latest['breach_prob_3m'] > 10:
            alerts.append(f"🔴 HIGH RISK: {latest['breach_prob_3m']:.1f}% breach probability")
        elif latest['breach_prob_3m'] > 5:
            alerts.append(f"🟡 ELEVATED: {latest['breach_prob_3m']:.1f}% breach probability")
        
        if latest['slope'] < -0.3:
            alerts.append(f"🟡 DETERIORATING: Slope {latest['slope']:.2f}/month")
        
        if alerts:
            for alert in alerts:
                print(f"  {alert}")
        else:
            print("  ✅ No alerts - metrics within normal ranges")
        
        print("="*70)
    
    def _sanitize_json(self, obj):
        """Replace non-finite floats so JSON stays valid."""
        if isinstance(obj, float):
            return obj if math.isfinite(obj) else None
        if isinstance(obj, list):
            return [self._sanitize_json(v) for v in obj]
        if isinstance(obj, dict):
            return {k: self._sanitize_json(v) for k, v in obj.items()}
        return obj

    def to_json(self, results: List[Dict], output_file: str = "ford_trigger_data.json"):
        """Export trigger results to JSON"""
        safe = self._sanitize_json(results)
        with open(output_file, 'w') as f:
            json.dump(safe, f, indent=2)
        print(f"\n✅ Data exported to {output_file}")


# Example usage
if __name__ == "__main__":
    print("="*70)
    print("FORD ABS TRIGGER ANALYSIS")
    print("="*70)
    
    # Initialize parser
    parser = FordABSParser(debug=True)
    
    # Parse all HTML files in current directory
    data = parser.parse_directory(directory="src/components", deal_name="Ford-2025")
    
    if data:
        # Calculate trigger metrics
        print("\n" + "="*70)
        print("CALCULATING TRIGGER METRICS")
        print("="*70)
        
        results = parser.calculate_triggers()
        
        # Print summary
        parser.print_trigger_summary(results)
        
        # Export to JSON
        parser.to_json(results, "ford_A_trigger_data.json")
        
        print("\n✅ Ready to import into dashboard!")
        print("Use ford_trigger_data.json with the React dashboard")
    else:
        print("\n❌ No data parsed - check your HTML files")



FORD ABS TRIGGER ANALYSIS
Found 3 HTML file(s)

Parsing: Ford-2025-A.html

📊 Extracted Metrics:
  ✅ month: 2025-11
  ✅ poolBalance: 1264722768.7
  ✅ totalDelinq: 0.91
  ✅ delinq61plus: 0.1358
  ✅ cumLossRatio: 0.1178
  ✅ trigger: 0.8
  ❌ numContracts: None
  ✅ transactionMonth: 9
✅ Successfully parsed!

Parsing: Ford-2025-B.html

📊 Extracted Metrics:
  ✅ month: 2025-11
  ✅ poolBalance: 1538447182.74
  ✅ totalDelinq: 0.52
  ✅ delinq61plus: 0.053
  ✅ cumLossRatio: 0.0018
  ✅ trigger: 0.8
  ❌ numContracts: None
  ✅ transactionMonth: 3
✅ Successfully parsed!

Parsing: Ford-2025-C.html

📊 Extracted Metrics:
  ✅ month: 2025-11
  ✅ poolBalance: 1096614974.63
  ✅ totalDelinq: 0.35
  ✅ delinq61plus: 0.0
  ✅ cumLossRatio: 0.0
  ✅ trigger: 0.8
  ❌ numContracts: None
  ✅ transactionMonth: 1
✅ Successfully parsed!

✅ Successfully parsed 3 file(s)

CALCULATING TRIGGER METRICS

TRIGGER ANALYSIS - LATEST MONTH
Deal: Ford-2025
Month: 2025-11
Transaction Month: 1

📊 CURRENT STATE:
  61+ Delinquency (R):